## Mask-RCNN for Bird Detection
In this notebook we use Mask-RCNN to crop images and keep a bounding box surrounding the birds in the images.

This notebook should be executed on Google Colab

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.81 MiB | 39.99 MiB/s, done.
Resolving deltas: 100% (570/570), done.


In [0]:
%cd Mask_RCNN

/content/Mask_RCNN


In [0]:
!pip install -r requirements.txt

     |████████████████████████████████| 163kB 2.8MB/s 
     |████████████████████████████████| 204kB 49.5MB/s 


In [0]:
from google.colab import files
uploaded = files.upload()

Saving bird_dataset_cropped.zip to bird_dataset_cropped (1).zip


In [0]:
!unzip bird_dataset_cropped.zip

Archive:  bird_dataset_cropped.zip
replace bird_dataset_cropped/test_images/mistery_category/002f61512a368e4c1434eedacf609957.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: bird_dataset_cropped/test_images/mistery_category/002f61512a368e4c1434eedacf609957.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/0267548c2aac82fe3d7e37ae98b00bd7.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/030c7d18b20ee586db3b74d9966c0348.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/034abbbb69336b0de7c7c0f2aa1267a6.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/035f7f6b1a162d86bb4755e6558f33d9.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/038872d6dfc340d0ff11f8d3c99d026e.jpg  
  inflating: bird_dataset_cropped/test_images/mistery_category/03b5b072b80ec04df1dee668f44abad4.jpg  
  inflating: 

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append("/samples/coco/")  # To find local version
print(sys.path)
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

/content
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content', '/content', '/content/samples/coco/', '/content', '/content/samples/coco/', '/content', '/content/samples/coco/', '/content', '/samples/coco/', '/content', '/samples/coco/']
... done downloading pretrained model!


In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [2]:
l = ['f003abbdf9329f5f631e4c040b9f0d3c.jpg']

In [0]:
from PIL import Image
listImage = []
## Create your own list of images you want to crop with l
for el in l:
    image = skimage.io.imread('bird_dataset_cropped/test_images/mistery_category/' + el)
    # Run detection

    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    if 15 in r['class_ids']:
    index = np.where(r['class_ids'] == 15)[0][0]
    image_PIL = Image.open('bird_dataset_cropped/test_images/mistery_category/' + el)
    x,y,z,d = r['rois'][index]
    image_PIL = image_PIL.crop( ( y, x, d, z) )
    image_PIL.save('bird_dataset_cropped/test_images/mistery_category/' + el, "JPEG", quality=80, optimize=True, progressive=True)

    #visualize.display_instances(listImage[1], r['rois'], r['masks'], r['class_ids'], 
                              #class_names, r['scores'])

Processing 1 images
image                    shape: (396, 440, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (332, 500, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (369, 500, 3)         min:    0.00000  max:  220.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  114.1

In [0]:
!zip -r birdZippedCropped.zip bird_dataset_cropped
files.download("birdZippedCropped.zip")

  adding: bird_dataset_cropped/ (stored 0%)
  adding: bird_dataset_cropped/test_images/ (stored 0%)
  adding: bird_dataset_cropped/test_images/mistery_category/ (stored 0%)
  adding: bird_dataset_cropped/test_images/mistery_category/5aab3de502e7501c9189d17cd0ca928b.jpg (deflated 4%)
  adding: bird_dataset_cropped/test_images/mistery_category/3151f1530e7c0cb6196e494c15eb493b.jpg (deflated 3%)
  adding: bird_dataset_cropped/test_images/mistery_category/3828cdb69f9ce6a1fad0e65c746f83cd.jpg (deflated 1%)
  adding: bird_dataset_cropped/test_images/mistery_category/8064ad6e67047772008ef4320d1515d4.jpg (deflated 3%)
  adding: bird_dataset_cropped/test_images/mistery_category/70fddb3c64b44a4d43e32a7c8148c425.jpg (deflated 5%)
  adding: bird_dataset_cropped/test_images/mistery_category/64a7a824d6d5c6dba0d881c4505e4240.jpg (deflated 8%)
  adding: bird_dataset_cropped/test_images/mistery_category/9b81acdecd60aee4738f9e2fce9d1d37.jpg (deflated 5%)
  adding: bird_dataset_cropped/test_images/mistery

MessageError: ignored